# GEMS PedTools Example Usage
Below is a simple example that illustrates how to access data in the PedTools database.
### Set up an HTTP client using Python's request library
We use a `Session` object to store our API key and automatically include it in the header for each request.

Note that we have a `api_key.py` file in the Exchange-Notebooks directory. The file contains only the below line.
```
api_key = 'SECRET'
```

In [1]:
import json
import pandas as pd
from requests import Session
from urllib.parse import quote
import sys
sys.path.append('..')
from api_key import api_key

s = Session()
s.headers.update({'apikey': api_key})

base = 'https://exchange-1.gems.msi.umn.edu/pedtools/v1'
# base = 'https://exchange-dev.gems.msi.umn.edu/pedtools/v1'

### Find info on a variety of interest
We use the `/{variety}` endpoint to obtain all recorded info on a variety, in this case 'TURKEY'.

In [2]:
params = {'pedigree_depth': 5}
variety = quote('TURKEY')
res = s.get(f'{base}/{variety}', params=params)
df = pd.json_normalize(res.json())
df

,preferred_name,crop_name,backcross_str,selfing_count,market_class,release_date,developer,parentage,aliases,father,...,mother.preferred_name,mother.crop_name,mother.backcross_str,mother.selfing_count,mother.market_class,mother.release_date,mother.developer,mother.parentage,mother.aliases,mother
0,TURKEY GID:135,wheat,,2,None,None,None,None,"[{'name': '135', 'type': 'germplasm_bank_id'},...",None,...,P.1066-? GID:3700,wheat,,1.0,NaN,NaN,NaN,NaN,"[{'name': '3700', 'type': 'germplasm_bank_id'}...",NaN
1,TURKEY GID:10509,wheat,,0,None,None,None,None,"[{'name': '10509', 'type': 'germplasm_bank_id'...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cool two items were returned. Can I see all the aliases for the first one?

In [3]:
df['aliases'][1]

[{'name': '10509', 'type': 'germplasm_bank_id'},
 {'name': 'TURKEY', 'type': 'bcid'},
 {'name': 'TURKEY 13.R', 'type': 'cross_name'}]

### Let's check another entry, and retrieve its pedigree to a depth of 5 (great-great-great grandparents).

In [4]:
params = {'pedigree_depth': 5}
variety = quote('SANDPIPER')
#variety = quote('###33476')
res = s.get(f'{base}/{variety}', params=params)
df = pd.json_normalize(res.json())
df

,preferred_name,crop_name,backcross_str,selfing_count,market_class,release_date,developer,parentage,aliases,pedigree,...,mother.aliases,father.preferred_name,father.crop_name,father.backcross_str,father.selfing_count,father.market_class,father.release_date,father.developer,father.parentage,father.aliases
0,SANDPIPER,wheat,,0,None,None,None,None,"[{'name': '4046', 'type': 'germplasm_bank_id'}...",###123561 / STEWART 63 /2/ 980 GID:980 / WELLS...,...,"[{'name': '1132', 'type': 'germplasm_bank_id'}...",FLAMINGO,wheat,,6,None,None,None,None,"[{'name': '-B14263.05-0T-0L-28T-0L-?', 'type':..."


### One match. Let's retrieve its pedigree.

In [5]:
df['pedigree'][0]

'###123561 / STEWART 63 /2/ 980 GID:980 / WELLS /3/ ###123633 / TEHUACAN 60 /2/ 980 GID:980 / WELLS /4/ POLONICUM PI185309 /2/ ###250171 / TEHUACAN 60 /3/ ###123633 / TEHUACAN 60 /2/ 980 GID:980 / WELLS /5/ II23584 GID:417 (II23584) F6 /2/ 3852 (###6385) F2 /3/ 3855 (###3926) F1 (###7220) F6'

### How do I find a matrix of Coefficient of Parentage (COP) values among any arbitrary list of varieties?
We use the post request `cop/matrix` endpoint to obtain the COP matrix as follows. Note that names of varieties do *not* need to be quote escaped since they will be in the body of the post not in the URL.

In [6]:
params = {'max_depth': 20}
var_list = ['FLAMINGO', 'SANDPIPER', 'GAVIOTA', 'TURKEY']
res = s.post(f'{base}/cop/matrix', params=params, json=var_list)
res.json()

{'reverse_mapping': {'FLAMINGO': 'FLAMINGO',
  'GAVIOTA': 'GAVIOTA',
  'SANDPIPER': 'SANDPIPER',
  'TURKEY GID:135': 'TURKEY',
  'TURKEY GID:10509': 'TURKEY'},
 'forward_mapping': {'FLAMINGO': ['FLAMINGO'],
  'GAVIOTA': ['GAVIOTA'],
  'SANDPIPER': ['SANDPIPER'],
  'TURKEY': ['TURKEY GID:135', 'TURKEY GID:10509']},
 'cop': {'FLAMINGO': {'FLAMINGO': 0.9921875,
   'GAVIOTA': 0.0,
   'SANDPIPER': 0.49609375,
   'TURKEY GID:135': 0.0,
   'TURKEY GID:10509': 0.0},
  'GAVIOTA': {'FLAMINGO': 0.0,
   'GAVIOTA': 0.5,
   'SANDPIPER': 0.25,
   'TURKEY GID:135': 0.0,
   'TURKEY GID:10509': 0.0},
  'SANDPIPER': {'FLAMINGO': 0.49609375,
   'GAVIOTA': 0.25,
   'SANDPIPER': 0.5,
   'TURKEY GID:135': 0.0,
   'TURKEY GID:10509': 0.0},
  'TURKEY GID:135': {'FLAMINGO': 0.0,
   'GAVIOTA': 0.0,
   'SANDPIPER': 0.0,
   'TURKEY GID:135': 0.875,
   'TURKEY GID:10509': 0.0},
  'TURKEY GID:10509': {'FLAMINGO': 0.0,
   'GAVIOTA': 0.0,
   'SANDPIPER': 0.0,
   'TURKEY GID:135': 0.0,
   'TURKEY GID:10509': 0.5}}}